In [32]:
import pandas as pd
import os

df = pd.read_csv('../data/02_limpia/df_cleaned.csv')
df

,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas
0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,2000,1,9,10101,6,0,0,0,0,0,0
1,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,2000,1,36,10101,10,0,0,0,0,0,0
2,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,2000,2,9,10101,6,0,0,0,0,0,0
3,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,2000,2,36,10101,7,3,0,0,0,0,0
4,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,2000,3,36,10101,8,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137726,UCAYALI,PURUS,PURUS,2023,48,25,250401,41,0,0,0,0,0,0
2137727,UCAYALI,PURUS,PURUS,2023,49,25,250401,51,0,0,0,0,0,0
2137728,UCAYALI,PURUS,PURUS,2023,50,25,250401,55,0,0,0,0,0,0
2137729,UCAYALI,PURUS,PURUS,2023,51,25,250401,36,0,0,0,0,0,0


In [33]:
id = 52

In [34]:
df.shape


(2137731, 14)

In [35]:
district = "all"

# Filtrar el DataFrame por el distrito indicado

df.shape


(2137731, 14)

In [36]:
df.distrito.value_counts()

distrito
SANTA ROSA        11204
LIMA               7397
INDEPENDENCIA      6557
BELLAVISTA         6119
SAN LUIS           5716
                  ...  
CIELO PUNCO          67
UNION PROGRESO       52
PATIBAMBA            23
PUTIS                17
SAN JOAQUIN           1
Name: count, Length: 1734, dtype: int64

In [37]:
cantidad_cols_inicio = df.columns

In [38]:
import pandas as pd

# Crear carpeta 'completos' en Colab
output_dir = "/content/completos"
os.makedirs(output_dir, exist_ok=True)

id = 50
# Leer el archivo CSV
df = pd.read_csv(f"../data/sub_reg_nt_csvs/sub_reg_nt_{id}.csv")

# Ordenar por año y semana
df = df.sort_values(by=["ano", "semana"]).reset_index(drop=True)

# PASO 1: Encontrar el distrito con más semanas
print("PASO 1: Identificando distrito con más semanas")
print("="*60)

# Contar el número de semanas por distrito
conteo_semanas = df.groupby(['departamento', 'provincia', 'distrito']).size().reset_index(name='total_semanas')

# Encontrar el distrito con más semanas
distrito_max_semanas = conteo_semanas.loc[conteo_semanas['total_semanas'].idxmax()]

print("Distrito con más semanas:")
print(f"Departamento: {distrito_max_semanas['departamento']}")
print(f"Provincia: {distrito_max_semanas['provincia']}")
print(f"Distrito: {distrito_max_semanas['distrito']}")
print(f"Total de semanas: {distrito_max_semanas['total_semanas']}")

# PASO 2: Filtrar el DataFrame para el distrito seleccionado
print(f"\nPASO 2: Filtrando datos del distrito seleccionado")
print("="*60)

df_7_distrito = df[
    (df['departamento'] == distrito_max_semanas['departamento']) &
    (df['provincia'] == distrito_max_semanas['provincia']) &
    (df['distrito'] == distrito_max_semanas['distrito'])
].copy()

print(f"DataFrame 'df_7_distrito' creado con {len(df_7_distrito)} registros")
print(f"Rango de años: {df_7_distrito['ano'].min()} - {df_7_distrito['ano'].max()}")
print(f"Rango de semanas: {df_7_distrito['semana'].min()} - {df_7_distrito['semana'].max()}")

# PASO 3: Analizar completitud de semanas por año
print(f"\nPASO 3: Analizando completitud de semanas por año")
print("="*60)

semanas_por_ano = df_7_distrito.groupby('ano')['semana'].nunique().reset_index()
semanas_por_ano.columns = ['ano', 'semanas_registradas']

# Identificar años con todas las semanas completas (52 o 53 semanas)
anos_completos = semanas_por_ano[semanas_por_ano['semanas_registradas'] >= 52]

print("Análisis de completitud de semanas por año:")
print(semanas_por_ano.to_string(index=False))

if len(anos_completos) > 0:
    print(f"\nAños con todas las semanas completas (≥52 semanas):")
    print(anos_completos.to_string(index=False))

    print(f"\nRango de años con semanas completas:")
    print(f"Desde: {anos_completos['ano'].min()}")
    print(f"Hasta: {anos_completos['ano'].max()}")
    print(f"Total de años completos: {len(anos_completos)}")

    # Verificar si hay años consecutivos completos
    anos_consecutivos = []
    anos_lista = sorted(anos_completos['ano'].tolist())

    if len(anos_lista) > 1:
        inicio_consecutivo = anos_lista[0]
        fin_consecutivo = anos_lista[0]

        for i in range(1, len(anos_lista)):
            if anos_lista[i] == anos_lista[i-1] + 1:
                fin_consecutivo = anos_lista[i]
            else:
                if fin_consecutivo > inicio_consecutivo:
                    anos_consecutivos.append((inicio_consecutivo, fin_consecutivo))
                inicio_consecutivo = anos_lista[i]
                fin_consecutivo = anos_lista[i]

        # Agregar el último rango si es consecutivo
        if fin_consecutivo > inicio_consecutivo:
            anos_consecutivos.append((inicio_consecutivo, fin_consecutivo))

        if anos_consecutivos:
            print(f"\nRangos de años consecutivos con semanas completas:")
            for inicio, fin in anos_consecutivos:
                print(f"  {inicio} - {fin} ({fin-inicio+1} años consecutivos)")

    # PASO 4: Crear DataFrame final con años completos
    print(f"\nPASO 4: Creando DataFrame final con años completos")
    print("="*60)

    df_7_distrito_completo = df_7_distrito[df_7_distrito['ano'].isin(anos_completos['ano'])].copy()

    print(f"DataFrame filtrado con años completos creado:")
    print(f"Registros originales en df_7_distrito: {len(df_7_distrito)}")
    print(f"Registros en df_7_distrito_completo: {len(df_7_distrito_completo)}")
    print(f"Años incluidos: {sorted(df_7_distrito_completo['ano'].unique().tolist())}")

    # Verificar que efectivamente cada año tiene todas sus semanas
    print(f"\nVerificación de completitud en el DataFrame filtrado:")
    verificacion = df_7_distrito_completo.groupby('ano')['semana'].agg(['min', 'max', 'nunique']).reset_index()
    verificacion.columns = ['ano', 'semana_min', 'semana_max', 'total_semanas']
    print(verificacion.to_string(index=False))

    # Guardar en archivo CSV
    # Guardar en archivo CSV dentro de la carpeta
    archivo_salida = os.path.join(output_dir, f"distrito_semanas_completas{inicio}_{fin}_id_{id}.csv")
    df_7_distrito_completo.to_csv(archivo_salida, index=False)
    print(f"\nDataFrame guardado en: {archivo_salida}")

    # Mostrar estadísticas resumidas del DataFrame final
    print(f"\nEstadísticas del DataFrame final (df_7_distrito_completo):")
    print("="*60)
    print(f"Distrito: {df_7_distrito_completo['distrito'].iloc[0]}")
    print(f"Provincia: {df_7_distrito_completo['provincia'].iloc[0]}")
    print(f"Departamento: {df_7_distrito_completo['departamento'].iloc[0]}")
    print(f"Período: {df_7_distrito_completo['ano'].min()} - {df_7_distrito_completo['ano'].max()}")
    print(f"Total de años: {df_7_distrito_completo['ano'].nunique()}")
    print(f"Total de registros: {len(df_7_distrito_completo)}")

    # Mostrar las primeras filas del DataFrame final
    print(f"\nPrimeras 10 filas de df_7_distrito_completo:")
    print("="*50)
    print(df_7_distrito_completo.head(10))

else:
    print("\nNo se encontraron años con todas las semanas completas (≥52 semanas)")
    print("No se puede crear el DataFrame con años completos.")

PASO 1: Identificando distrito con más semanas
Distrito con más semanas:
Departamento: LIMA
Provincia: LIMA
Distrito: CHORRILLOS
Total de semanas: 1248

PASO 2: Filtrando datos del distrito seleccionado
DataFrame 'df_7_distrito' creado con 1248 registros
Rango de años: 2000 - 2023
Rango de semanas: 1 - 52

PASO 3: Analizando completitud de semanas por año
Análisis de completitud de semanas por año:
 ano  semanas_registradas
2000                   52
2001                   52
2002                   52
2003                   52
2004                   52
2005                   52
2006                   52
2007                   52
2008                   52
2009                   52
2010                   52
2011                   52
2012                   52
2013                   52
2014                   52
2015                   52
2016                   52
2017                   52
2018                   52
2019                   52
2020                   52
2021                   52


In [39]:
df.distrito.value_counts()

distrito
SAN LUIS                1259
LIMA                    1248
LA VICTORIA             1248
CHORRILLOS              1248
SAN MARTIN DE PORRES    1248
                        ... 
HUANTAN                    1
LEONCIO PRADO              1
SAN PEDRO DE PILAS         1
SAN MATEO DE OTAO          1
SAN JOAQUIN                1
Name: count, Length: 144, dtype: int64

In [40]:
# Mostrar todas las columnas sin truncar
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_colwidth", None)

In [41]:
df.shape

(54299, 14)

In [42]:
cantidad_cols_inicio = len(df.columns)

In [43]:
df.head()

,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas
0,LIMA,LIMA,LIMA,2000,1,50,150101,681,6,0,1,0,0,0
1,LIMA,LIMA,ANCON,2000,1,50,150102,5,0,0,0,0,0,0
2,LIMA,LIMA,ATE,2000,1,50,150103,88,2,0,2,0,0,0
3,LIMA,LIMA,BARRANCO,2000,1,50,150104,1,0,0,0,0,0,0
4,LIMA,LIMA,BRE�A,2000,1,50,150105,143,0,0,0,0,0,0


Crear mes

In [44]:
import numpy as np

# Definir los nombres de los meses
meses = [
    "Enero", "Febrero", "Marzo", "Abril",
    "Mayo", "Junio", "Julio", "Agosto",
    "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# Crear nueva columna 'mes' usando la semana
df["mes"] = df["semana"].apply(lambda x: meses[(x - 1) // 4] if x <= 48 else "Diciembre")

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes
0,LIMA,LIMA,LIMA,2000,1,50,150101,681,6,0,1,0,0,0,Enero
1,LIMA,LIMA,ANCON,2000,1,50,150102,5,0,0,0,0,0,0,Enero
2,LIMA,LIMA,ATE,2000,1,50,150103,88,2,0,2,0,0,0,Enero
3,LIMA,LIMA,BARRANCO,2000,1,50,150104,1,0,0,0,0,0,0,Enero
4,LIMA,LIMA,BRE�A,2000,1,50,150105,143,0,0,0,0,0,0,Enero


Crear mes anterior

In [45]:
import numpy as np

# Definir los nombres de los meses
meses = [
    "Enero", "Febrero", "Marzo", "Abril",
    "Mayo", "Junio", "Julio", "Agosto",
    "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# Crear nueva columna 'mes' usando la semana
df["tiempo_mes"] = df["semana"].apply(lambda x: meses[(x - 1) // 4] if x <= 48 else "Diciembre")

# Crear columna 'mes_anterior'
df["tiempo_mes_anterior"] = df["mes"].apply(
    lambda m: meses[(meses.index(m) - 1) % 12]
)

df.head(5)


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior
0,LIMA,LIMA,LIMA,2000,1,50,150101,681,6,0,1,0,0,0,Enero,Enero,Diciembre
1,LIMA,LIMA,ANCON,2000,1,50,150102,5,0,0,0,0,0,0,Enero,Enero,Diciembre
2,LIMA,LIMA,ATE,2000,1,50,150103,88,2,0,2,0,0,0,Enero,Enero,Diciembre
3,LIMA,LIMA,BARRANCO,2000,1,50,150104,1,0,0,0,0,0,0,Enero,Enero,Diciembre
4,LIMA,LIMA,BRE�A,2000,1,50,150105,143,0,0,0,0,0,0,Enero,Enero,Diciembre


In [46]:
def obtener_estacion(semana):
    if (semana >= 51 and semana <= 53) or (semana >= 1 and semana <= 12):
        return "Verano"
    elif 13 <= semana <= 25:
        return "Otoño"
    elif 26 <= semana <= 38:
        return "Invierno"
    elif 39 <= semana <= 50:
        return "Primavera"
    else:
        return "Desconocido"

# Crear la nueva columna 'estacion'
df["tiempo_estacion"] = df["semana"].apply(obtener_estacion)

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion
0,LIMA,LIMA,LIMA,2000,1,50,150101,681,6,0,1,0,0,0,Enero,Enero,Diciembre,Verano
1,LIMA,LIMA,ANCON,2000,1,50,150102,5,0,0,0,0,0,0,Enero,Enero,Diciembre,Verano
2,LIMA,LIMA,ATE,2000,1,50,150103,88,2,0,2,0,0,0,Enero,Enero,Diciembre,Verano
3,LIMA,LIMA,BARRANCO,2000,1,50,150104,1,0,0,0,0,0,0,Enero,Enero,Diciembre,Verano
4,LIMA,LIMA,BRE�A,2000,1,50,150105,143,0,0,0,0,0,0,Enero,Enero,Diciembre,Verano


In [47]:
# Orden de meses
meses = [
    "Enero", "Febrero", "Marzo", "Abril",
    "Mayo", "Junio", "Julio", "Agosto",
    "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# Crear un mapeo de mes → número
mes_to_num = {mes: i+1 for i, mes in enumerate(meses)}

# Convertir 'mes' a número de mes
df["mes_num"] = df["mes"].map(mes_to_num)

# Crear columnas de periodos
df["tiempo_bimestre"] = ((df["mes_num"] - 1) // 2) + 1   # 1-6
df["tiempo_trimestre"] = ((df["mes_num"] - 1) // 3) + 1  # 1-4
df["tiempo_cuatrimestre"] = ((df["mes_num"] - 1) // 4) + 1  # 1-3
df["tiempo_semestre"] = ((df["mes_num"] - 1) // 6) + 1   # 1-2

# Eliminar la columna auxiliar
df = df.drop(columns=["mes_num"])

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre
0,LIMA,LIMA,LIMA,2000,1,50,150101,681,6,0,1,0,0,0,Enero,Enero,Diciembre,Verano,1,1,1,1
1,LIMA,LIMA,ANCON,2000,1,50,150102,5,0,0,0,0,0,0,Enero,Enero,Diciembre,Verano,1,1,1,1
2,LIMA,LIMA,ATE,2000,1,50,150103,88,2,0,2,0,0,0,Enero,Enero,Diciembre,Verano,1,1,1,1
3,LIMA,LIMA,BARRANCO,2000,1,50,150104,1,0,0,0,0,0,0,Enero,Enero,Diciembre,Verano,1,1,1,1
4,LIMA,LIMA,BRE�A,2000,1,50,150105,143,0,0,0,0,0,0,Enero,Enero,Diciembre,Verano,1,1,1,1


In [48]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "ira_no_neumonia"
col_lag = f"lag_semana_anterior_{col_base}"

# Crear la columna desplazada por grupo de distrito
df[col_lag] = (
    df.groupby("distrito")[col_base]
      .shift(1)
      .fillna(0)  # en el primer caso ponemos 0
)

# Ajuste para los saltos de año (semana 1)
mask_salto_ano = (df["semana"] == 1)

for idx in df[mask_salto_ano].index:
    distrito = df.at[idx, "distrito"]
    ano_prev = df.at[idx, "ano"] - 1
    semana_prev = df.loc[
        (df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"
    ].max()

    if pd.notna(semana_prev):
        valor_prev = df.loc[
            (df["distrito"] == distrito) &
            (df["ano"] == ano_prev) &
            (df["semana"] == semana_prev),
            col_base
        ].values
        if len(valor_prev) > 0:
            df.at[idx, col_lag] = valor_prev[0]
        else:
            df.at[idx, col_lag] = 0
    else:
        df.at[idx, col_lag] = 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0


In [49]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "ira_no_neumonia"
col_lag2 = f"lag_2_semanas_{col_base}"

# Crear la columna desplazada (lag de 2 semanas)
df[col_lag2] = (
    df.groupby("distrito")[col_base]
      .shift(2)
      .fillna(0)  # primeras dos filas se ponen en 0
)

# Ajuste para los saltos de año (semana 1 y semana 2)
mask_salto_ano = df["semana"].isin([1, 2])

for idx in df[mask_salto_ano].index:
    distrito = df.at[idx, "distrito"]
    ano_prev = df.at[idx, "ano"] - 1
    semanas_prev = df.loc[(df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"]

    if not semanas_prev.empty:
        semanas_sorted = sorted(semanas_prev.unique())
        if df.at[idx, "semana"] == 1 and len(semanas_sorted) >= 2:
            semana_prev = semanas_sorted[-2]  # la penúltima semana del año anterior
        elif df.at[idx, "semana"] == 2:
            semana_prev = semanas_sorted[-1]  # la última semana del año anterior
        else:
            semana_prev = None

        if semana_prev:
            valor_prev = df.loc[
                (df["distrito"] == distrito) &
                (df["ano"] == ano_prev) &
                (df["semana"] == semana_prev),
                col_base
            ].values
            df.at[idx, col_lag2] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0


In [50]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "ira_no_neumonia"
col_lag3 = f"lag_3_semanas_{col_base}"

# Crear la columna desplazada (lag de 3 semanas)
df[col_lag3] = (
    df.groupby("distrito")[col_base]
      .shift(3)
      .fillna(0)  # primeras tres filas se ponen en 0
)

# Ajuste para los saltos de año (semanas 1, 2 y 3)
mask_salto_ano = df["semana"].isin([1, 2, 3])

for idx in df[mask_salto_ano].index:
    distrito = df.at[idx, "distrito"]
    ano_prev = df.at[idx, "ano"] - 1
    semanas_prev = df.loc[(df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"]

    if not semanas_prev.empty:
        semanas_sorted = sorted(semanas_prev.unique())
        semana_actual = df.at[idx, "semana"]

        if semana_actual == 1 and len(semanas_sorted) >= 3:
            semana_prev = semanas_sorted[-3]  # la antepenúltima semana del año anterior
        elif semana_actual == 2 and len(semanas_sorted) >= 2:
            semana_prev = semanas_sorted[-2]  # la penúltima semana del año anterior
        elif semana_actual == 3 and len(semanas_sorted) >= 1:
            semana_prev = semanas_sorted[-1]  # la última semana del año anterior
        else:
            semana_prev = None

        if semana_prev:
            valor_prev = df.loc[
                (df["distrito"] == distrito) &
                (df["ano"] == ano_prev) &
                (df["semana"] == semana_prev),
                col_base
            ].values
            df.at[idx, col_lag3] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0


In [51]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "ira_no_neumonia"
col_lag4 = f"lag_4_semanas_{col_base}"

# Crear la columna desplazada (lag de 4 semanas)
df[col_lag4] = (
    df.groupby("distrito")[col_base]
      .shift(4)
      .fillna(0)  # primeras cuatro filas se ponen en 0
)

# Ajuste para los saltos de año (semanas 1, 2, 3 y 4)
mask_salto_ano = df["semana"].isin([1, 2, 3, 4])

for idx in df[mask_salto_ano].index:
    distrito = df.at[idx, "distrito"]
    ano_prev = df.at[idx, "ano"] - 1
    semanas_prev = df.loc[(df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"]

    if not semanas_prev.empty:
        semanas_sorted = sorted(semanas_prev.unique())
        semana_actual = df.at[idx, "semana"]

        if semana_actual == 1 and len(semanas_sorted) >= 4:
            semana_prev = semanas_sorted[-4]  # cuarta semana antes de fin de año
        elif semana_actual == 2 and len(semanas_sorted) >= 3:
            semana_prev = semanas_sorted[-3]
        elif semana_actual == 3 and len(semanas_sorted) >= 2:
            semana_prev = semanas_sorted[-2]
        elif semana_actual == 4 and len(semanas_sorted) >= 1:
            semana_prev = semanas_sorted[-1]
        else:
            semana_prev = None

        if semana_prev:
            valor_prev = df.loc[
                (df["distrito"] == distrito) &
                (df["ano"] == ano_prev) &
                (df["semana"] == semana_prev),
                col_base
            ].values
            df.at[idx, col_lag4] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0


In [52]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "ira_no_neumonia"
col_lag5 = f"lag_5_semanas_{col_base}"

# Crear la columna desplazada (lag de 5 semanas)
df[col_lag5] = (
    df.groupby("distrito")[col_base]
      .shift(5)
      .fillna(0)  # primeras cinco filas se ponen en 0
)

# Ajuste para los saltos de año (semanas 1 a 5)
mask_salto_ano = df["semana"].isin([1, 2, 3, 4, 5])

for idx in df[mask_salto_ano].index:
    distrito = df.at[idx, "distrito"]
    ano_prev = df.at[idx, "ano"] - 1
    semanas_prev = df.loc[(df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"]

    if not semanas_prev.empty:
        semanas_sorted = sorted(semanas_prev.unique())
        semana_actual = df.at[idx, "semana"]

        if semana_actual == 1 and len(semanas_sorted) >= 5:
            semana_prev = semanas_sorted[-5]
        elif semana_actual == 2 and len(semanas_sorted) >= 4:
            semana_prev = semanas_sorted[-4]
        elif semana_actual == 3 and len(semanas_sorted) >= 3:
            semana_prev = semanas_sorted[-3]
        elif semana_actual == 4 and len(semanas_sorted) >= 2:
            semana_prev = semanas_sorted[-2]
        elif semana_actual == 5 and len(semanas_sorted) >= 1:
            semana_prev = semanas_sorted[-1]
        else:
            semana_prev = None

        if semana_prev:
            valor_prev = df.loc[
                (df["distrito"] == distrito) &
                (df["ano"] == ano_prev) &
                (df["semana"] == semana_prev),
                col_base
            ].values
            df.at[idx, col_lag5] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0


In [53]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "ira_no_neumonia"
col_lag6 = f"lag_6_semanas_{col_base}"

# Crear la columna desplazada (lag de 6 semanas)
df[col_lag6] = (
    df.groupby("distrito")[col_base]
      .shift(6)
      .fillna(0)  # primeras seis filas se ponen en 0
)

# Ajuste para los saltos de año (semanas 1 a 6)
mask_salto_ano = df["semana"].isin([1, 2, 3, 4, 5, 6])

for idx in df[mask_salto_ano].index:
    distrito = df.at[idx, "distrito"]
    ano_prev = df.at[idx, "ano"] - 1
    semanas_prev = df.loc[(df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"]

    if not semanas_prev.empty:
        semanas_sorted = sorted(semanas_prev.unique())
        semana_actual = df.at[idx, "semana"]

        if semana_actual == 1 and len(semanas_sorted) >= 6:
            semana_prev = semanas_sorted[-6]
        elif semana_actual == 2 and len(semanas_sorted) >= 5:
            semana_prev = semanas_sorted[-5]
        elif semana_actual == 3 and len(semanas_sorted) >= 4:
            semana_prev = semanas_sorted[-4]
        elif semana_actual == 4 and len(semanas_sorted) >= 3:
            semana_prev = semanas_sorted[-3]
        elif semana_actual == 5 and len(semanas_sorted) >= 2:
            semana_prev = semanas_sorted[-2]
        elif semana_actual == 6 and len(semanas_sorted) >= 1:
            semana_prev = semanas_sorted[-1]
        else:
            semana_prev = None

        if semana_prev:
            valor_prev = df.loc[
                (df["distrito"] == distrito) &
                (df["ano"] == ano_prev) &
                (df["semana"] == semana_prev),
                col_base
            ].values
            df.at[idx, col_lag6] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia,lag_6_semanas_ira_no_neumonia
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0,0.0


In [54]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "neumonias_men5"

# Crear lags de 1 a 6 semanas
for lag in range(1, 7):
    col_lag = f"lag_{lag}_semanas_{col_base}"
    df[col_lag] = (
        df.groupby("distrito")[col_base]
          .shift(lag)
          .fillna(0)  # primeras filas se ponen en 0
    )

    # Ajuste para saltos de año (semanas iniciales)
    mask_salto_ano = df["semana"].isin(range(1, lag + 1))

    for idx in df[mask_salto_ano].index:
        distrito = df.at[idx, "distrito"]
        ano_prev = df.at[idx, "ano"] - 1
        semanas_prev = df.loc[
            (df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"
        ]

        if not semanas_prev.empty:
            semanas_sorted = sorted(semanas_prev.unique())
            semana_actual = df.at[idx, "semana"]

            # Calcular la semana previa según el lag
            if len(semanas_sorted) >= lag:
                semana_prev = semanas_sorted[-lag + (semana_actual - 1)]
            else:
                semana_prev = None

            if semana_prev:
                valor_prev = df.loc[
                    (df["distrito"] == distrito) &
                    (df["ano"] == ano_prev) &
                    (df["semana"] == semana_prev),
                    col_base
                ].values
                df.at[idx, col_lag] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia,lag_6_semanas_ira_no_neumonia,lag_1_semanas_neumonias_men5,lag_2_semanas_neumonias_men5,lag_3_semanas_neumonias_men5,lag_4_semanas_neumonias_men5,lag_5_semanas_neumonias_men5,lag_6_semanas_neumonias_men5
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [55]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "neumonias_60mas"

# Crear lags de 1 a 6 semanas
for lag in range(1, 7):
    col_lag = f"lag_{lag}_semanas_{col_base}"
    df[col_lag] = (
        df.groupby("distrito")[col_base]
          .shift(lag)
          .fillna(0)  # primeras filas se ponen en 0
    )

    # Ajuste para saltos de año (semanas iniciales)
    mask_salto_ano = df["semana"].isin(range(1, lag + 1))

    for idx in df[mask_salto_ano].index:
        distrito = df.at[idx, "distrito"]
        ano_prev = df.at[idx, "ano"] - 1
        semanas_prev = df.loc[
            (df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"
        ]

        if not semanas_prev.empty:
            semanas_sorted = sorted(semanas_prev.unique())
            semana_actual = df.at[idx, "semana"]

            # Calcular la semana previa según el lag
            if len(semanas_sorted) >= lag:
                semana_prev = semanas_sorted[-lag + (semana_actual - 1)]
            else:
                semana_prev = None

            if semana_prev:
                valor_prev = df.loc[
                    (df["distrito"] == distrito) &
                    (df["ano"] == ano_prev) &
                    (df["semana"] == semana_prev),
                    col_base
                ].values
                df.at[idx, col_lag] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia,lag_6_semanas_ira_no_neumonia,lag_1_semanas_neumonias_men5,lag_2_semanas_neumonias_men5,lag_3_semanas_neumonias_men5,lag_4_semanas_neumonias_men5,lag_5_semanas_neumonias_men5,lag_6_semanas_neumonias_men5,lag_1_semanas_neumonias_60mas,lag_2_semanas_neumonias_60mas,lag_3_semanas_neumonias_60mas,lag_4_semanas_neumonias_60mas,lag_5_semanas_neumonias_60mas,lag_6_semanas_neumonias_60mas
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "hospitalizados_men5"

# Crear lags de 1 a 6 semanas
for lag in range(1, 7):
    col_lag = f"lag_{lag}_semanas_{col_base}"
    df[col_lag] = (
        df.groupby("distrito")[col_base]
          .shift(lag)
          .fillna(0)  # primeras filas del grupo en 0
    )

    # Ajuste para saltos de año (semanas iniciales de cada año)
    mask_salto_ano = df["semana"].isin(range(1, lag + 1))

    for idx in df[mask_salto_ano].index:
        distrito = df.at[idx, "distrito"]
        ano_prev = df.at[idx, "ano"] - 1
        semanas_prev = df.loc[
            (df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"
        ]

        if not semanas_prev.empty:
            semanas_sorted = sorted(semanas_prev.unique())
            semana_actual = df.at[idx, "semana"]

            # Calcular la semana previa según el lag
            if len(semanas_sorted) >= lag:
                semana_prev = semanas_sorted[-lag + (semana_actual - 1)]
            else:
                semana_prev = None

            if semana_prev:
                valor_prev = df.loc[
                    (df["distrito"] == distrito) &
                    (df["ano"] == ano_prev) &
                    (df["semana"] == semana_prev),
                    col_base
                ].values
                df.at[idx, col_lag] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia,lag_6_semanas_ira_no_neumonia,lag_1_semanas_neumonias_men5,lag_2_semanas_neumonias_men5,lag_3_semanas_neumonias_men5,lag_4_semanas_neumonias_men5,lag_5_semanas_neumonias_men5,lag_6_semanas_neumonias_men5,lag_1_semanas_neumonias_60mas,lag_2_semanas_neumonias_60mas,lag_3_semanas_neumonias_60mas,lag_4_semanas_neumonias_60mas,lag_5_semanas_neumonias_60mas,lag_6_semanas_neumonias_60mas,lag_1_semanas_hospitalizados_men5,lag_2_semanas_hospitalizados_men5,lag_3_semanas_hospitalizados_men5,lag_4_semanas_hospitalizados_men5,lag_5_semanas_hospitalizados_men5,lag_6_semanas_hospitalizados_men5
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [57]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "hospitalizados_60mas"

# Crear lags de 1 a 6 semanas
for lag in range(1, 7):
    col_lag = f"lag_{lag}_semanas_{col_base}"
    df[col_lag] = (
        df.groupby("distrito")[col_base]
          .shift(lag)
          .fillna(0)  # primeras filas del grupo en 0
    )

    # Ajuste para saltos de año (semanas iniciales de cada año)
    mask_salto_ano = df["semana"].isin(range(1, lag + 1))

    for idx in df[mask_salto_ano].index:
        distrito = df.at[idx, "distrito"]
        ano_prev = df.at[idx, "ano"] - 1
        semanas_prev = df.loc[
            (df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"
        ]

        if not semanas_prev.empty:
            semanas_sorted = sorted(semanas_prev.unique())
            semana_actual = df.at[idx, "semana"]

            # Calcular la semana previa según el lag
            if len(semanas_sorted) >= lag:
                semana_prev = semanas_sorted[-lag + (semana_actual - 1)]
            else:
                semana_prev = None

            if semana_prev:
                valor_prev = df.loc[
                    (df["distrito"] == distrito) &
                    (df["ano"] == ano_prev) &
                    (df["semana"] == semana_prev),
                    col_base
                ].values
                df.at[idx, col_lag] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia,lag_6_semanas_ira_no_neumonia,lag_1_semanas_neumonias_men5,lag_2_semanas_neumonias_men5,lag_3_semanas_neumonias_men5,lag_4_semanas_neumonias_men5,lag_5_semanas_neumonias_men5,lag_6_semanas_neumonias_men5,lag_1_semanas_neumonias_60mas,lag_2_semanas_neumonias_60mas,lag_3_semanas_neumonias_60mas,lag_4_semanas_neumonias_60mas,lag_5_semanas_neumonias_60mas,lag_6_semanas_neumonias_60mas,lag_1_semanas_hospitalizados_men5,lag_2_semanas_hospitalizados_men5,lag_3_semanas_hospitalizados_men5,lag_4_semanas_hospitalizados_men5,lag_5_semanas_hospitalizados_men5,lag_6_semanas_hospitalizados_men5,lag_1_semanas_hospitalizados_60mas,lag_2_semanas_hospitalizados_60mas,lag_3_semanas_hospitalizados_60mas,lag_4_semanas_hospitalizados_60mas,lag_5_semanas_hospitalizados_60mas,lag_6_semanas_hospitalizados_60mas
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
import pandas as pd

# Ordenamos bien el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "defunciones_men5"

# Crear lags de 1 a 6 semanas
for lag in range(1, 7):
    col_lag = f"lag_{lag}_semanas_{col_base}"
    df[col_lag] = (
        df.groupby("distrito")[col_base]
          .shift(lag)
          .fillna(0)  # primeras filas del grupo en 0
    )

    # Ajuste para saltos de año (semanas iniciales de cada año)
    mask_salto_ano = df["semana"].isin(range(1, lag + 1))

    for idx in df[mask_salto_ano].index:
        distrito = df.at[idx, "distrito"]
        ano_prev = df.at[idx, "ano"] - 1
        semanas_prev = df.loc[
            (df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"
        ]

        if not semanas_prev.empty:
            semanas_sorted = sorted(semanas_prev.unique())
            semana_actual = df.at[idx, "semana"]

            # Calcular la semana previa según el lag
            if len(semanas_sorted) >= lag:
                semana_prev = semanas_sorted[-lag + (semana_actual - 1)]
            else:
                semana_prev = None

            if semana_prev:
                valor_prev = df.loc[
                    (df["distrito"] == distrito) &
                    (df["ano"] == ano_prev) &
                    (df["semana"] == semana_prev),
                    col_base
                ].values
                df.at[idx, col_lag] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia,lag_6_semanas_ira_no_neumonia,lag_1_semanas_neumonias_men5,lag_2_semanas_neumonias_men5,lag_3_semanas_neumonias_men5,lag_4_semanas_neumonias_men5,lag_5_semanas_neumonias_men5,lag_6_semanas_neumonias_men5,lag_1_semanas_neumonias_60mas,lag_2_semanas_neumonias_60mas,lag_3_semanas_neumonias_60mas,lag_4_semanas_neumonias_60mas,lag_5_semanas_neumonias_60mas,lag_6_semanas_neumonias_60mas,lag_1_semanas_hospitalizados_men5,lag_2_semanas_hospitalizados_men5,lag_3_semanas_hospitalizados_men5,lag_4_semanas_hospitalizados_men5,lag_5_semanas_hospitalizados_men5,lag_6_semanas_hospitalizados_men5,lag_1_semanas_hospitalizados_60mas,lag_2_semanas_hospitalizados_60mas,lag_3_semanas_hospitalizados_60mas,lag_4_semanas_hospitalizados_60mas,lag_5_semanas_hospitalizados_60mas,lag_6_semanas_hospitalizados_60mas,lag_1_semanas_defunciones_men5,lag_2_semanas_defunciones_men5,lag_3_semanas_defunciones_men5,lag_4_semanas_defunciones_men5,lag_5_semanas_defunciones_men5,lag_6_semanas_defunciones_men5
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
import pandas as pd

# Ordenamos el DataFrame por distrito, año y semana
df = df.sort_values(by=["distrito", "ano", "semana"]).reset_index(drop=True)

col_base = "defunciones_60mas"

# Crear lags de 1 a 6 semanas
for lag in range(1, 7):
    col_lag = f"lag_{lag}_semanas_{col_base}"
    df[col_lag] = (
        df.groupby("distrito")[col_base]
          .shift(lag)
          .fillna(0)  # primeras filas del grupo en 0
    )

    # Ajuste para saltos de año (semanas iniciales de cada año)
    mask_salto_ano = df["semana"].isin(range(1, lag + 1))

    for idx in df[mask_salto_ano].index:
        distrito = df.at[idx, "distrito"]
        ano_prev = df.at[idx, "ano"] - 1
        semanas_prev = df.loc[
            (df["distrito"] == distrito) & (df["ano"] == ano_prev), "semana"
        ]

        if not semanas_prev.empty:
            semanas_sorted = sorted(semanas_prev.unique())
            semana_actual = df.at[idx, "semana"]

            # Calcular la semana previa según el lag
            if len(semanas_sorted) >= lag:
                semana_prev = semanas_sorted[-lag + (semana_actual - 1)]
            else:
                semana_prev = None

            if semana_prev:
                valor_prev = df.loc[
                    (df["distrito"] == distrito) &
                    (df["ano"] == ano_prev) &
                    (df["semana"] == semana_prev),
                    col_base
                ].values
                df.at[idx, col_lag] = valor_prev[0] if len(valor_prev) > 0 else 0

df.head()


,departamento,provincia,distrito,ano,semana,sub_reg_nt,ubigeo,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,mes,tiempo_mes,tiempo_mes_anterior,tiempo_estacion,tiempo_bimestre,tiempo_trimestre,tiempo_cuatrimestre,tiempo_semestre,lag_semana_anterior_ira_no_neumonia,lag_2_semanas_ira_no_neumonia,lag_3_semanas_ira_no_neumonia,lag_4_semanas_ira_no_neumonia,lag_5_semanas_ira_no_neumonia,lag_6_semanas_ira_no_neumonia,lag_1_semanas_neumonias_men5,lag_2_semanas_neumonias_men5,lag_3_semanas_neumonias_men5,lag_4_semanas_neumonias_men5,lag_5_semanas_neumonias_men5,lag_6_semanas_neumonias_men5,lag_1_semanas_neumonias_60mas,lag_2_semanas_neumonias_60mas,lag_3_semanas_neumonias_60mas,lag_4_semanas_neumonias_60mas,lag_5_semanas_neumonias_60mas,lag_6_semanas_neumonias_60mas,lag_1_semanas_hospitalizados_men5,lag_2_semanas_hospitalizados_men5,lag_3_semanas_hospitalizados_men5,lag_4_semanas_hospitalizados_men5,lag_5_semanas_hospitalizados_men5,lag_6_semanas_hospitalizados_men5,lag_1_semanas_hospitalizados_60mas,lag_2_semanas_hospitalizados_60mas,lag_3_semanas_hospitalizados_60mas,lag_4_semanas_hospitalizados_60mas,lag_5_semanas_hospitalizados_60mas,lag_6_semanas_hospitalizados_60mas,lag_1_semanas_defunciones_men5,lag_2_semanas_defunciones_men5,lag_3_semanas_defunciones_men5,lag_4_semanas_defunciones_men5,lag_5_semanas_defunciones_men5,lag_6_semanas_defunciones_men5,lag_1_semanas_defunciones_60mas,lag_2_semanas_defunciones_60mas,lag_3_semanas_defunciones_60mas,lag_4_semanas_defunciones_60mas,lag_5_semanas_defunciones_60mas,lag_6_semanas_defunciones_60mas
0,LIMA,YAUYOS,ALLAUCA,2021,47,50,151003,1,0,0,0,0,0,0,Diciembre,Diciembre,Noviembre,Primavera,6,4,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LIMA,YAUYOS,ALLAUCA,2021,52,50,151003,0,1,0,1,0,0,0,Diciembre,Diciembre,Noviembre,Verano,6,4,3,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LIMA,YAUYOS,ALLAUCA,2022,43,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LIMA,YAUYOS,ALLAUCA,2022,44,50,151003,0,0,0,0,0,0,0,Noviembre,Noviembre,Octubre,Primavera,6,4,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LIMA,YAUYOS,ALLAUCA,2023,14,50,151003,0,0,1,0,0,0,0,Abril,Abril,Marzo,Otoño,2,2,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
cantidad_cols_fin = len(df.columns)

In [61]:
cantidad_cols_fin - cantidad_cols_inicio

50

In [62]:
import os

# Carpeta de salida
carpeta = "../data/03_casos"
os.makedirs(carpeta, exist_ok=True)

# Nombre del archivo de salida
archivo_salida = os.path.join(carpeta, f"df_con_variables_sub_reg_nt_{id}-{district}.csv")

# Guardar el DataFrame
df.to_csv(archivo_salida, index=False, encoding="utf-8-sig")

print(f"✅ DataFrame guardado en: {archivo_salida}")


✅ DataFrame guardado en: ../data/03_casos\df_con_variables_sub_reg_nt_50-all.csv
